In [42]:
from twitter_keys import consumer_key, consumer_secret, access_token, access_secret
import tweepy #for twitter API
import pickle
import pandas as pd # To Handle Data
import numpy as np #for number computing
from collections import Counter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from textblob import TextBlob 
from wordcloud import WordCloud, STOPWORDS 
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\s524702\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
MeToo_tweets=[]
for tweet in tweepy.Cursor(api.search, q='#MeToo',lang='en', tweet_mode='extended', ).items(1000):
    MeToo_tweets.append(tweet)
#with open("MeToo.pkl", 'wb') as f:
    #pickle.dump(MeToo_tweets, f)
with open('MeToo.pkl', 'rb') as f:
    MeToo_tweets = pickle.load(f)

In [40]:
data = pd.DataFrame(data=[tweet.full_text for tweet in MeToo_tweets], columns=['MeTooTweets'])
data['MeToolen']  = np.array([len(tweet.full_text) for tweet in MeToo_tweets])
data['MeTooDate'] = np.array([tweet.created_at for tweet in MeToo_tweets])
data['MeTooSource'] = np.array([tweet.source for tweet in MeToo_tweets])
data['MeTooLikes']  = np.array([tweet.favorite_count for tweet in MeToo_tweets])
data['MeTooRTs']    = np.array([tweet.retweet_count for tweet in MeToo_tweets])
display(data.head(20))

,MeTooTweets,MeToolen,MeTooDate,MeTooSource,MeTooLikes,MeTooRTs
0,Hey! Hey! @gkhowe29 &amp; I are super passiona...,260,2019-03-07 14:53:55,Twitter for iPhone,0,0
1,Pretty much. #YesAllWomen #MeToo https://t.co/...,56,2019-03-07 14:53:54,Zoho Social,0,0
2,RT @grantstern: Gayle King had to put up with ...,139,2019-03-07 14:53:53,Twitter for iPhone,0,16
3,Where is the #MeToo #TimeUp movement right now...,82,2019-03-07 14:53:47,Twitter Web Client,0,0
4,RT @grantstern: Gayle King had to put up with ...,139,2019-03-07 14:53:40,Twitter for iPhone,0,16
5,RT @IreneWoodbury: Jen &amp; Colton + Brandi &...,148,2019-03-07 14:53:39,Twitter Web Client,0,81
6,#cancelplanetfitness #metoo #WomensRights #fit...,163,2019-03-07 14:53:24,Twitter for iPhone,0,0
7,Sorry #putney made Dan Franklin @JonathanCape ...,300,2019-03-07 14:53:14,Twitter for iPhone,0,0
8,RT @nighatdad: #WhyIMarch because when women s...,140,2019-03-07 14:52:44,Twitter for Android,0,42
9,@taylornation13 “Coming forward is an agonizin...,289,2019-03-07 14:52:29,Twitter for iPhone,0,0


In [11]:
MeToolexemes = []
for tweet in MeToo_tweets:
    MeToolexemes.extend(tweet.full_text.split())
def lexical_diversity(tokens):
    return len(set(tokens))/len(tokens)


In [41]:
positive=0
negative=0
neutral=0
for tweet in MeToo_tweets:
    analysis = TextBlob(tweet.full_text)
    if analysis.sentiment[0]>0:
       positive+=1
    elif analysis.sentiment[0]<0:
       negative+=1
    else:
      neutral+=1
df_bar= pd.DataFrame({'lab':['Positive', 'Negative', 'Neutral'], 'val':[positive, negative, neutral]})
ax = df_bar.plot.bar(x='lab', y='val', rot=0)

In [63]:
#wordCloud for #meToo
words = data.MeTooTweets[0]
stopwords = set(STOPWORDS)
stopwords.update(["gkhowe29", "amp", "metoo", "https", "V9GHLyRRXQ","co"])
wordcloud = WordCloud(stopwords=stopwords,background_color="red").generate(words)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()